# Part-1: Data Processing

Данные: https://www.dropbox.com/s/s4qj0fpsn378m2i/chgk.zip 

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
* взять в тренировочный набор турниры с dateStart из 2019 года; 
* в тестовый — турниры с dateStart из 2020 года.

In [22]:
import pickle
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.svm import LinearSVR

In [2]:
# source: https://stackoverflow.com/questions/19201290/how-to-save-a-dictionary-to-a-file/32216025

def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
TRAIN_MODE = False

### Достаем ids турниров (2019 -- train, 2020 -- test)

In [24]:
df_tournaments = pd.DataFrame(pd.read_pickle('chgk/tournaments.pkl')).transpose()
df_tournaments = df_tournaments[df_tournaments.dateStart >= '2019-01-01']

tournaments_ids_all = df_tournaments[df_tournaments.dateStart >= '2019-01-01']
tournaments_ids_all = set(tournaments_ids_all['id'])
save_obj(tournaments_ids_all, 'tournaments_ids_all')

tournaments_ids_test = df_tournaments[df_tournaments.dateStart >= '2020-01-01']
tournaments_ids_test = set(tournaments_ids_test['id'])
save_obj(tournaments_ids_test, 'tournaments_ids_test')

tournaments_ids_train = tournaments_ids_all.difference(tournaments_ids_test)
save_obj(tournaments_ids_train, 'tournaments_ids_train')

len(tournaments_ids_all), len(tournaments_ids_train), len(tournaments_ids_test)

(1109, 687, 422)

### Среди всех турниров оставляем только турниры:
* нужных лет (2019-2020);
* с mask для всех участников (повопросные ответы)
* с teamMembers для всех участников (данные об участниках)

In [25]:
def get_results_df(tournament_ids):
    df_results = pd.read_pickle('chgk/results.pkl')
    print("full dataframe length = ", len(df_results))
    results_all = {}
    for k, v in df_results.items():
        # игнорируем турниры до 2019 года, а также пустые записи
        if k in tournament_ids and len(v) > 0:
            valid = True
            # игнорируем турниры, где нет нужных нам валидных полей
            for team_data in v:
                if 'team' not in team_data or 'mask' not in team_data or 'teamMembers' not in team_data:
                    valid = False
                    continue
                if team_data['mask'] is None or team_data['team'] is None or team_data['teamMembers'] is None:
                    valid = False
                    continue
            if valid:
                results_all[k] = v
    print("cleared dataframe length = ", len(results_all))
    return results_all

df_test = get_results_df(tournaments_ids_test)
save_obj(df_test, 'test')

full dataframe length =  5528
cleared dataframe length =  169


### Преобразуем в датафрейм ('tournament_id', 'team_id', 'player_id', 'mask')

In [26]:
def unwrap_player(df):
    df_results_cleaned = []
    for k, v in df.items():
        for team_data in v:
            team = team_data['team']
            mask = str(team_data['mask']).replace('X', '0').replace('?', '0')
            players = team_data['teamMembers']
            for player in players:
                df_results_cleaned.append([k, team['id'], player['player']['id'], mask])
    df = pd.DataFrame(df_results_cleaned)
    df.columns = ['tournament_id', 'team_id', 'player_id', 'mask']
    return df

df_train = get_results_df(tournaments_ids_train)
df_train = unwrap_player(df_train)

full dataframe length =  5528
cleared dataframe length =  671


### Преобразуем в датафрейм ('tournament_id', 'team_id', 'player_id', 'question_local_id', 'target')
#### Замечание: для этого разворачиваем mask -> (question_local_id, target)

In [27]:
def unwrap_mask(df):
    df_results_cleaned = []
    for _, row in df.iterrows():
        tt_id = row['tournament_id']
        tm_id = row['team_id']
        pr_id = row['player_id']
        mask = row['mask']
        for idx in range(len(mask)):
            df_results_cleaned.append([tt_id, tm_id, pr_id, idx, mask[idx]])
    df = pd.DataFrame(df_results_cleaned)
    df.columns = ['tournament_id', 'team_id', 'player_id', 'question_local_id', 'target']
    return df

df_train = unwrap_mask(df_train)
save_obj(df_train, 'train')

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:

повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.

### Схема baseline:

C помощью логистической регрессии будем предсказывать $z_{i,j}$ -- вероятность ответил ли игрок i на вопрос в рамках турнира j:
* Преобразуем 'tournament_id', 'player_id' в one-hot-фичи, 'team_id' и 'question_local_id' не используем;
* Обучаем logreg на целевой переменной target (ответил / не ответил на вопрос данного турнира);
* Сохраняем финальные веса фичей пользователей (one-hot из player_id), вес пользователя интерпретируем как его "силу", и за счет ранжирования весов получаем рейтинг

In [28]:
if TRAIN_MODE:
    # get preprocessed df ('tournament_id', 'team_id', 'player_id', 'question_local_id', 'target')
    df = pd.read_csv('train.zip').drop(columns=['team_id', 'question_local_id'])
    
    # construct pipeline with one-hot-encoder and logreg
    feature_generation = ColumnTransformer(
        transformers=[
            ('OneHot', OneHotEncoder(), ['tournament_id', 'player_id'])
        ],
        remainder='drop',
        sparse_threshold=1
    )
    pipe = Pipeline(
        verbose=True,
        steps=[
            ('feature_generation', feature_generation),
            ('classifier', LogisticRegression(solver='liblinear', max_iter=100))
        ]
    )
    
    # train
    pipe.fit(df[['tournament_id', 'player_id']], df['target'])
    
    # save player weights (~raiting)
    player_features_start_pos = df.nunique()['tournament_id']
    player_features_names = pipe['feature_generation'].get_feature_names()[player_features_start_pos:]
    assert(len(player_features_names) == df.nunique()['player_id'])
    player_ids = [int(name[11:]) for name in player_features_names]
    
    player_weights = pipe['classifier'].coef_[0][player_features_start_pos:]
    assert(len(player_weights) == df.nunique()['player_id'])
    
    player_to_weight = dict(zip(player_ids, player_weights))
    save_obj(player_to_weight, 'player_to_weight')

[Pipeline]  (step 1 of 2) Processing feature_generation, total=  11.0s
[Pipeline] ........ (step 2 of 2) Processing classifier, total=35.7min


Грубая проверка: сравним top-100 официального рейтинга и предсказания
Официальный рейтинг: https://rating.chgk.info/players.php

In [29]:
if not TRAIN_MODE:
    player_to_weight = load_obj('player_to_weight')

In [30]:
official_top_100_ids = pd.read_csv('players-official-top-1000.csv')[:100]
official_top_100_ids = set(official_top_100_ids[' ИД'])

FileNotFoundError: [Errno 2] No such file or directory: 'players-official-top-1000.csv'

In [31]:
player_to_weight_sorted = sorted(player_to_weight.items(), key=lambda kv: kv[1], reverse=True)
predicted_top_100_ids = set(k for k, v in player_to_weight_sorted[:100])

In [32]:
len(official_top_100_ids.intersection(predicted_top_100_ids))

NameError: name 'official_top_100_ids' is not defined

### Модель смогла предсказать больше 53 игрока из топ 100.
#### Т.е грубая проверка показывает, что большие веса получили сильные игроки. Основная валидация на тестовых данных в Part-3

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
* предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
* в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

Для самопроверки: у Сергея средняя корреляция Спирмена на тестовом множестве 2020 года во всех моделях, включая baselines, получалась порядка 0.7-0.8, а корреляция Кендалла — порядка 0.5-0.6. Если у корреляции вышли за 0.9 или, наоборот, упали ниже 0.3, скорее всего где-то баг.

#### * Вес (сила) команды = вес ее участников (их обучили на трейне)
#### * В рамках одного турнира предлагается ранжировать команды по их весу

In [33]:
player_to_weight = load_obj('player_to_weight')

In [34]:
df_test = load_obj('test')

In [35]:
def get_positions_label(tournament):
    return [team['position'] for team in tournament]

def get_position_prediction(tournament):
    """
    ранжируем команды по весу = (сумма весов участников),
    есть игрока не было в train -- берем средний вес игрока в трейне
    """
    avg_weight = np.mean([v for v in player_to_weight.values()])
    team_rating = []
    for idx, team in enumerate(tournament):
        weight = 0
        for player_info in team['teamMembers']:
            p_id = player_info['player']['id']
            try:
                weight += player_to_weight[p_id]
            except:
                weight += avg_weight
        team_rating.append((idx + 1, weight))
    team_rating = sorted(team_rating, key=lambda kv: kv[1], reverse=True)
    return [pos for pos, weight in team_rating]

In [36]:
def get_score(df, corr):
    x = [corr(get_positions_label(t), get_position_prediction(t)).correlation for t in df.values()]
    x = np.array(x)
    x = x[~np.isnan(x)]
    return np.mean(x)

for corr in [('Spearman', stats.spearmanr), ('Kendall ', stats.kendalltau)]:
    print(f'Avg {corr[0]} corr value for df = {get_score(df_test, corr[1])}')

Avg Spearman corr value for df = 0.7821800575848492
Avg Kendall  corr value for df = 0.6252692792756139


# Part-4: EM algorithm

Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
* предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
* разработайте EM-схему для обучения этой модели, реализуйте её в коде;
* обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


# Решение

### M-step
* В рамках baseline (part 2) мы научились предсказывать $p(z_{i,j}=1)$ -- вероятность ответил ли игрок i на вопрос j, веса фичей пользователя обученной модели мы интерпретировали как их "силу";
* Если выбрать $z_{i, j}$ в качестве скрытых переменных, то M-шаг сводится к дообучению модели при заданных $z_(i, j)$, начальные веса совпадают с исходными метками $x_{i,j}$

In [37]:
# data: describe and computed in part 1
n_epoch = 5
df_train = pd.read_csv('train.zip')
df_train["question_id"] = df_train['tournament_id'].astype(str) + '_' + df_train['question_local_id'].astype(str)
df_train = df_train.drop(columns=['tournament_id', 'question_local_id'])
X, y = df_train[['player_id', 'question_id']], df_train['target']
df_train.head()

,team_id,player_id,target,question_id
0,45556,6212,1,4772_0
1,45556,6212,1,4772_1
2,45556,6212,1,4772_2
3,45556,6212,1,4772_3
4,45556,6212,1,4772_4


In [38]:
# m-step model: described in part 2

feature_generation = ColumnTransformer(
    transformers=[('OneHot', OneHotEncoder(), ['player_id', 'question_id'])],
    remainder='drop',
    sparse_threshold=1
)

# Замечание: пришлось заменить sklearn::LogisticRegression, тк в ходе выполнения задания
# выяснилось, что она плохо работает с небинарными таргетами, поэтому заменил ее на другой регрессор:
# https://stackoverflow.com/questions/47663569/how-to-do-regression-as-opposed-to-classification-using-logistic-regression-and
pipe = Pipeline(
    verbose=True,
    steps=[
        ('feature_generation', feature_generation),
        ('regressor', LinearSVR(loss='squared_epsilon_insensitive'))
    ]
)

def m_step(model, X, y):
    model.fit(X, y)
    return model, model.predict(X)

In [39]:
def save_player_weights(X, model):
    """
    сохраняем веса фичей пользователей из обученного классификатора
    """
    player_features_end_pos = X.nunique()['question_id']
    player_features_names = model['feature_generation'].get_feature_names()[0:player_features_end_pos]
    player_ids = [int(name[11:]) for name in player_features_names]
    player_weights = model['regressor'].coef_[0:player_features_end_pos]
    player_to_weight = dict(zip(player_ids, player_weights))
    return player_to_weight

#_, preds = m_step(pipe, X, y)
#validate(load_obj('test'), save_player_weights(X, _))

# E-step

* Теперь хотим учесть наличие команды, тч $z_{i,j}$ стали зависимыми для игроков одной команды, перейдем к прогнозированию условных вероятностей $p(z_{i,j} = 1)$ -> $p(z_{i,j} = 1 | team_{i, j} = 1)$, где $team_{i, j}$ -- ответила ли команда игрока i на вопрос j;
* Предположим, что $$team_{i, j} = 1 \iff \exists k \in team_i : z_{k, j} = 1$$
* И наоборот: $$team_{i, j} = 0 \iff \forall k \in team_i : z_{k, j} = 0$$

По теореме Байеса имеем:
$$
    p(z_{i,j}=1|team_{i,j}=1) = \frac{p(team_{i,j}=1|z_{i,j}=1) p(z_{i,j}=1)}{p(team_{i,j}=1)}
$$

C учетом предположений имеем:
$$
    p(z_{i,j}=1|team_{i,j}=1) = \frac{p(z_{i,j}=1)}{1 - p(team_{i,j}=0)} = \frac{p(z_{i,j}=1)}{1 - \Pi_{k \in team_i} \left(1 - p(z_{k,j}=1)\right)}
$$

С учетом того, что $p(z_{i,j}=1)$ являются результатом M-шага, то формула выше может быть использована для E-шага

In [40]:
def e_step(df, preds):
    df['new_target'] = preds
    label_zero_idx = df['target'] == 0
    df.loc[label_zero_idx, 'new_target'] = 0
    # изменяем только метки для вопросов, на которые команда ответила
    # поскольку p(z_ij = 1 | team_ij = 0) = 0 в силу предположений
    label_one_idx = df['target'] == 1
    e_step_denom = df.loc[label_one_idx].groupby(['team_id', 'question_id'])['new_target']
    e_step_denom = e_step_denom.transform(lambda x : 1 - np.prod(1 - x.values))
    df.loc[label_one_idx, 'new_target'] = df.loc[label_one_idx, 'new_target'] / e_step_denom
    new_y = df['new_target'].fillna(0)
    return new_y

In [ ]:
# initialization
pipe, preds = m_step(pipe, X, y)
#validate(load_obj('test'), save_player_weights(X, pipe))

In [ ]:
# EM-iterations
for i in range(n_epoch):
    y = e_step(df_train, preds)
    pipe, preds = m_step(pipe, X, y)
    weights = save_player_weights(X, pipe)
    save_obj(weights, f'em_weights_epoch_{i}')

In [ ]:
# validation: described in part 3

def get_positions_label(tournament):
    """
    позиции команд в турнире (фактические)
    """
    return [team['position'] for team in tournament]


def get_position_prediction(tournament, player_to_weight):
    """
    позиции команд в турнире (предсказанные),
    ранжируем команды по весу = (сумма весов участников),
    есть игрока не было в train -- берем средний вес игрока в трейне
    """
    avg_weight = np.mean([v for v in player_to_weight.values()])
    team_rating = []
    for idx, team in enumerate(tournament):
        weight = 0
        for player_info in team['teamMembers']:
            p_id = player_info['player']['id']
            try:
                weight += player_to_weight[p_id]
            except:
                weight += avg_weight
        team_rating.append((idx + 1, weight))
    team_rating = sorted(team_rating, key=lambda kv: kv[1], reverse=True)
    return [pos for pos, weight in team_rating]


def get_score(df_test, player_to_weight, corr):
    """
    среднее значение rank correlation по тестовой выборке
    """
    x = [corr(get_positions_label(t), get_position_prediction(t, player_to_weight)).correlation for t in df_test.values()]
    x = np.array(x)
    x = x[~np.isnan(x)]
    return np.mean(x)

def validate(df_test, player_to_weight, corr=None):
    if corr is None:
        for corr in [('Spearman', stats.spearmanr), ('Kendall ', stats.kendalltau)]:
            print(f'Avg {corr[0]} corr value for df = {get_score(df_test, player_to_weight, corr[1])}')
    else:
        print(f'Avg {corr[0]} corr value for df = {get_score(df_test, player_to_weight, corr[1])}')

In [ ]:
for i in range(n_epoch):
    print(f'Epoch {i+1}:')
    validate(load_obj('test'), load_obj(f'em_weights_epoch_{i}'))

# Метрики не растут. В чем дело? Ход разбирательства:

* Бейзлайн модель LogisticRegression, метрики падали сильнейшим образом, выяснил что данная модель из sklearn некорректно работает с небинарными таргетами, а именно они возникают на итерациях EM-алгоритм;
* Заменил модель на LinearSVR, настроил параметры: метрики все также не растут, но хотя бы перестали убывать;
* Заменил фичи m-step модели: (tournament_id, player_id) -> (player_id, question_id), по рост скачком выглядит подозрительно